<a href="https://colab.research.google.com/github/4en1x/diploma/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install gcc-4.8 g++-4.8 -y -qq;
!ln -s /usr/bin/gcc-4.8 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-4.8 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-4.8 60 --slave /usr/bin/g++ g++ /usr/bin/g++-4.8

26 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2019-02-25 12:45:09--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?_faoX_VkFeKDcugf9BZoJcu-y76xJOONESZ5mJfksQcHei0q-1FzLqgXp9WmELfOhQxdf0Vshp5ZxDTPlaSbpABtscJIZc7fGIKa-goDyDKcHjjkbcsae5iaFQ5Hgor3TVQHCUQBQqoJZnxvsa9RkhMtpMS7WrrH6NmxcyqiSoSRv5la-buE98lstqgW4Q6JvW9-4-cvn00Jeo9r--2rtpIsHQ [following]
--2019-02-25 12:45:10--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?_faoX_VkFeKDcugf9BZoJc

In [0]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2016 NVIDIA Corporation
Built on Tue_Jan_10_13:22:03_CST_2017
Cuda compilation tools, release 8.0, V8.0.61


In [0]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-9_w_j947
  Stored in directory: /tmp/pip-ephem-wheel-cache-77szfhym/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [0]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [0]:
!cd /usr/src
!ln -sfn linux-headers-2.6.35-28-generic linux

CODE (AT LEAST NOW)

In [0]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [0]:
%%cu
#include <stdio.h>

int main() {
	int deviceCount;
  cudaDeviceProp devProp;
    
  cudaGetDeviceCount(&deviceCount);
  printf("Found %d devices\n", deviceCount);
	return 0;
}


'Found 1 devices\n'

In [58]:
%%cu
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include <stdlib.h>
#include <stdio.h>
#include "linux/kernel.h"
#include <iostream>
#include <ctime>
#include <iomanip>
#include <fstream>
#include <math.h>
#include <cmath>

using namespace std;

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort = true) {
	if (code != cudaSuccess) {
		fprintf(stderr, "GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
		if (abort) {
			exit(code);
		}
	}
}

#define ITERATIONS 100
#define N 2048

#define e 2.71
#define Dx 1
#define h ((float) Dx / N)
#define h2 powf(h, 2)

#define max(x, y) (((x) > (y)) ? (x) : (y))
#define min(x, y) (((x) < (y)) ? (x) : (y))

#define r1 16
#define r2 16

#define t1 2
#define t2 2

__device__
float f(int i, int j) {
	return powf(e, i * h + j * h);
}

float u_t(float x) {
	return powf(e, x);
}

float u_l(float x) {
	return powf(e, x);
}

float u_r(float x) {
	return powf(e, x + 1);
}

float u_d(float x) {
	return powf(e, x + 1);
}


__device__
int u(float* matrix, int i, int j) {
	return matrix[N*i + j];
}

__device__
void tile(float* matrix, int iGl, int jGl, bool isBlack) {
	int iBegin = iGl * r1 + threadIdx.x * t1;
	int jBegin = jGl * r2 + threadIdx.y * t2;

	for (size_t i = max(iBegin, 1); i < min(iBegin + t1, N - 1); i++) {
		int offset = i % 2;
		if (isBlack) {
			offset = 1 - offset;
		}
		for (size_t j = max(jBegin + offset, 1); j < min(jBegin + t2, N - 1); j+=2) {
			matrix[N*i + j] = 0.25 * (u(matrix, i + 1, j)
				+ u(matrix, i - 1, j)
				+ u(matrix, i, j + 1)
				+ u(matrix, i, j - 1) - h2 * f(i, j));
		}
	}
}

__global__
void calculateBlack(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, true);
}

__global__
void calculateRed(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, false);
}

float func(int i, int j) {
	return powf(e, i * h + j * h);
}

float* createMatrix(int row, int col) {
	float* matrix = new float[row * col];
	for(size_t i = 0; i < row; i++) {
		for(size_t j = 0; j < col; j++) {
			matrix[i * row + j] = 0;
		}
	}
    
  for(size_t i = 1; i < row - 1; i++) {
    matrix[0 * row + i] = u_t(i * h);
    matrix[(row - 1) * row + i] = u_d(i * h);
	}

  for(size_t j = 1; j < col - 1; j++) {
    matrix[j * col + 0] = u_l(j * h);
    matrix[j * col + row - 1] = u_r(j * h);
	}
    
	return matrix;
}

float calculateTolerance(float* matrix1, float* matrix2) {
	float tolerance = 0;
	for (size_t i = 0; i < N * N; i++) {
		tolerance += (matrix1[i] - matrix2[i]) * (matrix1[i] - matrix2[i]);
	}
	return tolerance;
}

int main() {
  ofstream myfile;
  myfile.open("example.txt");
	float* matrixHost = createMatrix(N, N);
  
/*
  for (size_t i = 0; i < N; i++) {
		for (size_t j = 0; j < N; j++) {
      myfile << matrixHost[i * N + j] << " ";
		}
    myfile << "\n";
	}
*/
  
	float* previousMatrixHost = new float[N * N];
	float* matrixDevice;
	
	int matrixSize = sizeof(float) * N * N;

	gpuErrchk(cudaMalloc((void**)&matrixDevice, matrixSize));
	gpuErrchk(cudaMemcpy(matrixDevice, matrixHost, matrixSize, cudaMemcpyHostToDevice));

	long startTime = clock();
	dim3 blocks(N / r1, N / r2);
	dim3 threads(r1 / t1, r2 / t2);

	for (size_t i = 0; i < ITERATIONS; i++) {
		gpuErrchk(cudaMemcpy(previousMatrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost));
		calculateBlack <<<blocks, threads >>>(matrixDevice);
		gpuErrchk(cudaDeviceSynchronize());
		calculateRed <<<blocks, threads >>>(matrixDevice);
		gpuErrchk(cudaPeekAtLastError());
		gpuErrchk(cudaDeviceSynchronize());
		gpuErrchk(cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost));
    myfile << calculateTolerance(matrixHost, previousMatrixHost) << "\n";
	}
	std::cout << ITERATIONS << " iteration norm: " << std::setprecision(8) << std::fixed << calculateTolerance(matrixHost, previousMatrixHost) << std::endl;
	std::cout << "Calculation take time: " << clock() - startTime << std::endl;
	cudaError_t cudaResult = cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

	if (cudaResult != cudaSuccess) {
		std::cout << "Cuda error: " << cudaResult << std::endl;
	}
  myfile.close();
	std::system("pause");
	return 0;
}


'100 iteration norm: 1.50000000\nCalculation take time: 2199317\nsh: 1: pause: not found\n'

In [59]:
import os

f = open("example.txt", 'r')
content = f.readlines()
for line in content:
  print(line)


6733.12

369.125

2.25

1.75

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

1.5

